In [5]:
%load_ext autoreload
%autoreload 2
#Set root directory
import sys
import os
sys.path.append(os.path.abspath(".."))

# Use gpu
import torch 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', device)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
using device:  cuda


In [6]:
from senior_thesis.helpers.researchRunner import ResearchRunner
from senior_thesis.agents.chunking.sentenceChunkingAgent import SentenceChunkingAgent
from senior_thesis.agents.embedding.sbertEmbeddingAgent import SBERTEmbeddingAgent
from senior_thesis.agents.clustering.kmeans import KMeansClusteringAgent
from senior_thesis.agents.clustering.hdbscan import HDBSCANClusteringAgent
import sqlite3


In [7]:
##Trial 1: Sentence Chunking + SBERT + KMeans (25 centers)
sentenceChunker = SentenceChunkingAgent()
sbert = SBERTEmbeddingAgent(device=device)
kmeans = KMeansClusteringAgent(100)

rr = ResearchRunner('test_db', 'sentence_sbert_kmeans_100', sentenceChunker, sbert, kmeans)


<pre style="font-family: Courier; margin:2px; padding-left:20px; line-height:1.2;">✅ ResearchRunner Initialized!</pre>

In [8]:
import umap
from senior_thesis.helpers.embeddings import Embeddings
from senior_thesis.agents.clustering.hdbscan import HDBSCANClusteringAgent
from senior_thesis.helpers.stopwatch import Stopwatch
stopwatch = Stopwatch()
conn = sqlite3.connect('test_db')
embeddings = Embeddings(conn, rr._get_embedding_table_name())

umapModel = umap.UMAP(n_components=50, metric='cosine', transform_mode='embedding')      
all_embeddings = embeddings.getAllEmbeddings(max=50000)
print(f"Grabbed {len(all_embeddings)} embeddings")
reduced_embeddings = umapModel.fit_transform(all_embeddings)
print(f"Finished in {stopwatch.measure()} seconds")

Grabbed 50000 embeddings
Finished in 0 min 35 sec seconds


In [9]:
from senior_thesis.helpers.stopwatch import Stopwatch
stopwatch = Stopwatch()
hdbscan = HDBSCANClusteringAgent(umap_model=umapModel, reduced_embeddings=reduced_embeddings)
hdbscan.pass_embeddings(embeddings)
hdbscan.train()
print(f"Done in {stopwatch.measure()} seconds")

Done in 0 min 20 sec seconds


In [12]:
stats = hdbscan.getStats()

In [1]:
print(stats)

NameError: name 'stats' is not defined